In [41]:
import sys
import numpy as np
sys.path.append('../')
from model import data_preparation as prep

In [42]:
con = prep.create_connection()
prepared_data = prep.prepare_data(con)

In [43]:
prepared_data

{88: {'abahn': False,
  'brand': 'shell',
  'bstr': True,
  'county': '01054',
  'sstr': False,
  'state': 'schleswig-holstein',
  'time_series': {'test': array([1349, 1339, 1329, ..., 1399, 1489, 1469]),
   'train': array([1399, 1399, 1399, ..., 1369, 1349, 1349])}},
 827: {'abahn': False,
  'brand': 'shell',
  'bstr': True,
  'county': '020000000000',
  'sstr': False,
  'state': 'hamburg',
  'time_series': {'test': array([1329, 1329, 1329, ..., 1309, 1489, 1489]),
   'train': array([1399, 1399, 1399, ..., 1329, 1329, 1329])}},
 1217: {'abahn': False,
  'brand': 'avia',
  'bstr': False,
  'county': '03461',
  'sstr': False,
  'state': 'niedersachsen',
  'time_series': {'test': array([1339, 1339, 1339, ..., 1339, 1339, 1339]),
   'train': array([1289, 1289, 1289, ..., 1349, 1339, 1339])}},
 2348: {'abahn': False,
  'brand': 'freie tankstelle',
  'bstr': True,
  'county': '03454',
  'sstr': False,
  'state': 'niedersachsen',
  'time_series': {'test': array([1429, 1429, 1429, ..., 1359, 

In [44]:
for key in prepared_data:
    prepared_data[key]["time_series"]["test"] = np.log(prepared_data[key]["time_series"]["test"].reshape(-1,1))
    prepared_data[key]["time_series"]["train"] = np.log(prepared_data[key]["time_series"]["train"].reshape(-1,1))
    

In [45]:
from statsmodels.tsa.stattools import pacf

keys = list(prepared_data.keys())

x_pacf=pacf( prepared_data[keys[0]]['time_series']['train'] , nlags =10, method='ols')

In [46]:
x_pacf

array([ 1.        ,  0.91620121,  0.2204104 ,  0.01167805, -0.02256453,
       -0.02539637, -0.0216598 ,  0.01725517,  0.05777609,  0.07587236,
        0.10204178])

In [47]:

from nnet_ts import *
count = 0
ahead =12

In [48]:
def train_net_on_station(trainset, ahead=12, count = 0):
    x = trainset
    x=x.reshape ( -1)
    while(count < ahead):
        end =len(x)-ahead+count
        np.random.seed(2016)
        fit1 = TimeSeriesNnet(hidden_layers =[7,3],activation_functions = [ "tanh" , "tanh" ] )
        fit1.fit(x[0:end],lag =1,epochs = 100)


        out = fit1.predict_ahead(n_ahead = 1)
        print ( " Obs : " , count +1, " x = " , round( x[count] , 4 ) , " prediction =", pd.Series(out))

        count = count + 1
        return fit1
    
train_net_on_station(prepared_data[keys[0]]['time_series']['train'])

INFO:Building regressor matrix
INFO:Scaling data
INFO:Checking network consistency
INFO:Training neural net
/usr/lib/python3.6/site-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


 Obs :  1  x =  7.2435  prediction = 0    7.178169
dtype: float64


In [49]:
def encodeState(data):
    states = {
     'baden-württemberg':1,
     'bayern':2,
     'berlin':3,
     'brandenburg':4,
     'bremen':5,
     'hamburg':6,
     'hessen':7,
     'mecklenburg-vorpommern':8,
     'niedersachsen':9,
     'nordrhein-westfalen':10,
     'rheinland-pfalz':11,
     'saarland':12,
     'sachsen':13,
     'sachsen-anhalt':14,
     'schleswig-holstein':15,
     'thüringen':16
    }
    data["state"] = states[data["state"]]
    return data


In [104]:
x_train_adjusted = {}
x_test_adjusted = {}
y_train_adjusted = {}
y_test_adjusted = {}
for k in keys:
    #prepared_data[k] = encodeState(prepared_data[k])
    x_train = prepared_data[k]["time_series"]["train"][0:-2]
    y_train_adjusted[k] = prepared_data[k]["time_series"]["train"][1:-1]
    x_test = prepared_data[k]["time_series"]["test"][0:-2]
    y_test_adjusted[k] = prepared_data[k]["time_series"]["test"][1:-1]
    x_train_adjusted[k] = []
    x_test_adjusted[k] = []
    for p in x_train:
        features = []
        features.append(p[0])
        features.append(prepared_data[k]["state"])
        features.append(prepared_data[k]["county"])
        features.append(int(prepared_data[k]["abahn"]))
        features.append(int(prepared_data[k]["bstr"]))
        features.append(int(prepared_data[k]["sstr"]))
        x_train_adjusted[k].append(features)
    x_test_adjusted[k] = []
    for p in x_test:
        features = []
        features.append(p[0])
        features.append(prepared_data[k]["state"])
        features.append(prepared_data[k]["county"])
        features.append(int(prepared_data[k]["abahn"]))
        features.append(int(prepared_data[k]["bstr"]))
        features.append(int(prepared_data[k]["sstr"]))
        x_test_adjusted[k].append(features)

In [105]:
y_train_adjusted[88]

array([[ 7.24351297],
       [ 7.24351297],
       [ 7.24351297],
       ..., 
       [ 7.22183583],
       [ 7.22183583],
       [ 7.20711886]])

In [106]:
from keras.optimizers import SGD
from keras.layers.core import Dense, Activation
from keras.layers.recurrent import SimpleRNN

In [107]:
seed = 2016
np.random.seed(seed)
fit1 = Sequential()
fit1.add(SimpleRNN(output_dim=8, activation="tanh", input_shape=(6, 1)))
fit1.add(Dense(output_dim =1, activation='linear'))
sgd = SGD(lr =0.0001, momentum =0.95, nesterov=True)
fit1.compile(loss='mean_squared_error', optimizer=sgd)

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `SimpleRNN` call to the Keras 2 API: `SimpleRNN(activation="tanh", input_shape=(6, 1), units=8)`
  after removing the cwd from sys.path.
/usr/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="linear", units=1)`
  """


In [114]:
x_train_adjusted[4515] = np.array(x_train_adjusted[4515])
x_test_adjusted[4515] = np.array(x_test_adjusted[4515])
x_train_adjusted[4515] = x_train_adjusted[4515].reshape(x_train_adjusted[4515].shape + (1, ))
x_test_adjusted[4515] = x_test_adjusted[4515].reshape(x_test_adjusted[4515].shape + (1, ))

In [116]:
fit1.fit(x_train_adjusted[88], y_train_adjusted[88], batch_size=10, epochs =10)


Epoch 1/10
19025/19025 [==============================] - 4s 236us/step - loss: 0.0051
Epoch 2/10
19025/19025 [==============================] - 5s 238us/step - loss: 0.0051
Epoch 3/10
19025/19025 [==============================] - 4s 229us/step - loss: 0.0051
Epoch 4/10
19025/19025 [==============================] - 4s 217us/step - loss: 0.0051
Epoch 5/10
19025/19025 [==============================] - 4s 216us/step - loss: 0.0051
Epoch 6/10
19025/19025 [==============================] - 4s 218us/step - loss: 0.0051
Epoch 7/10
19025/19025 [==============================] - 4s 222us/step - loss: 0.0051
Epoch 8/10
19025/19025 [==============================] - 4s 216us/step - loss: 0.0051
Epoch 9/10
19025/19025 [==============================] - 4s 228us/step - loss: 0.0051
Epoch 10/10
19025/19025 [==============================] - 5s 240us/step - loss: 0.0051


In [117]:
score_train = fit1.evaluate(x_train_adjusted[4515], y_train_adjusted[4515], batch_size =10)
score_test = fit1.evaluate (x_test_adjusted[4515], y_test_adjusted[4515], batch_size =10)
print("in train MSE = ", round( score_train, 6))
print("in test MSE = ", round( score_test ,6))

4755/4755 [==============================] - 1s 116us/step
in train MSE =  0.00413
in test MSE =  0.001098
